# Fast api 进行前后端分离 💬

目前我们已经完成了基本的可视化页面，并且可以实现对应的功能。

为了方便整个项目的管理，现有的项目通常采用前后端分离的方式搭建，前后端数据通过 json 的格式进行传输。


FastAPI 是一个用于构建 API 的现代、快速（高性能）的 web 框架，非常方便用于搭建我们的前后端分离的应用。

我们首先需要将我们用到的后端函数进行 FastAPI 的封装。封装 API 与前文中讲过将大模型 API 封装成本地 API 的方法类似，我们首先导入第三方库并创建一个 API 对象：

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import os

app = FastAPI() # 创建 api 对象

本地 API 一般通过 POST 方式进行访问，即参数会附加在 POST 请求中，我们需要定义一个数据模型来接收 POST 请求中的数据：

In [ ]:
# 定义一个数据模型，用于接收POST请求中的数据
class Item(BaseModel):
    prompt : str # 用户 prompt
    model : str = "gpt-3.5-turbo"# 使用的模型
    temperature : float = 0.1# 温度系数
    if_history : bool = False # 是否使用历史对话功能
    # API_Key
    api_key: str = None
    # Secret_Key
    secret_key : str = None
    # access_token
    access_token: str = None
    # APPID
    appid : str = None
    # APISecret
    api_secret : str = None
    # 数据库路径
    db_path : str = "../../data_base/vector_db/chroma"
    # 源文件路径
    file_path : str = "../../data_base/knowledge_db"
    # prompt template
    prompt_template : str = template
    # Template 变量
    input_variables : list = ["context","question"]
    # Embdding
    embedding : str = "openai"
    # Top K
    top_k : int = 5
    # embedding_key
    embedding_key : str = api_key

在上面的类中，我们定义了要调用我们已封装的 QA_chain 所需要传入的参数，对于非必须参数，我们都设置了默认参数来保证调用的简洁性，接下来我们就可以创建一个 POST 请求的 API 端点：

In [ ]:
@app.post("/answer/")
async def get_response(item: Item):

    # 首先确定需要调用的链
    if not item.if_history:
        # 调用 Chat 链
        chain = QA_chain_self(model=item.model, temperature=item.temperature, top_k=item.top_k, file_path=item.file_path, persist_path=item.db_path, 
                                appid=item.appid, api_key=item.api_key, embedding=item.embedding, template=template, api_secret=item.api_secret, embedding_key=item.embedding_key)

        response = chain.answer(question = item.prompt)
    
        return response
    
    # 由于 API 存在即时性问题，不能支持历史链
    else:
        return "API 不支持历史链"

上述端点的业务逻辑很简单，即调用我们已封装的 QA_chain_self 对象进行实例化与回答即可。通过这一个端点启动，我们便可通过访问本地 8000 端口来调用个人知识库助手的服务啦，我们只需要通过下列命令启动： 

In [ ]:
uvicorn app:app 

完整项目代码请查阅 [project/serve](/project/serve/) 目录~